In [128]:
import requests
from bs4 import BeautifulSoup
import bs4
import re 

books = {}
for i in range(60,100):
    if i < 70:
        URL = 'https://darktranslation.com/rezero-kara-hajimeru-isekai-seikatsu-arc-{arc}-bab-{chapter}/'.format(chapter=i, arc=4)
    elif i >= 70:
        URL = 'https://darktranslation.com/re-zero-kara-hajimeru-isekai-seikatsu-arc-{arc}-bab-{chapter}/'.format(chapter=i, arc=4)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')

    texts = soup.find_all(lambda tag: not tag.find(['span']) and tag.name == 'p' and 
                                       tag.get('style') == 'text-align: justify;', text=True)

    if len(texts) < 10:
        continue

    li = []
    for text in texts:
       li.append(text.text)
    
    books[i] = li

In [192]:
parent_URL = 'https://darktranslation.com/novel/baca-web-novel-rezero-indonesia/'

page = requests.get(parent_URL)

soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all(lambda tag: tag.name == 'a' and "Bab" in tag.text , href=re.compile("darktranslation.com"))

li = []

for link in links:
   li.append(link['href'])

chapters = {}
for link in li:
    chapterName = link.split('/')[-2]
    
    URL = link
    
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')

    texts = soup.find_all(lambda tag: not tag.find(['span']) and tag.name == 'p' and 
                                       tag.get('style') == 'text-align: justify;', text=True)

    if len(texts) < 10:
        continue

    temp = []
    for text in texts:
       temp.append(text.text)
    
    chapters[chapterName] = temp

In [208]:
import pickle 

chapterPkl = open("chapters.pkl", "wb")
pickle.dump(chapters, chapterPkl)
chapterPkl.close()

In [241]:
parent_URL = 'https://www.readlightnovel.org/novel-list'

page = requests.get(parent_URL)

soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all('a', {"data-toggle": "popover"}, href=re.compile("readlightnovel.org"))

li = []

for link in links:
   li.append(link['href'])

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [246]:
manga = {}

for link in li:
    
    mangaName= link.split('/')[-1]
# parent_URL = 'https://www.readlightnovel.org/novel-list'

    page = requests.get(link)

    soup = BeautifulSoup(page.content, 'html.parser')

    uls = soup.find_all(class_='chapter-chs')

    links = []
    for ul in uls:
        for tag in ul.findAll('a'):
            links.append(tag['href'])
            
    manga[mangaName] = links

In [250]:
mangaPkl = open("manga.pkl", "wb")
pickle.dump(manga, mangaPkl)
mangaPkl.close()

In [273]:
lightNovelData = {}
for mangaName, mangaChapters in manga.items():
    lightNovelData[mangaName] = {}
    
    for mangaLink in mangaChapters:
        page = requests.get(mangaLink)

        soup = BeautifulSoup(page.content, 'html.parser')

        texts = soup.find_all(lambda tag: not tag.find(['strong']) and tag.name == 'p', text=True)

        temp = []
        for text in texts:
           temp.append(text.text)
        
        chapterName = '.'.join(mangaLink.split('/')[3:])

        lightNovelData[mangaName][chapterName] = temp

KeyboardInterrupt: 